# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:

df = data[['race','call','firstname']]
print(df.head())
print(df.info())


  race  call firstname
0    w   0.0   Allison
1    w   0.0   Kristen
2    b   0.0   Lakisha
3    b   0.0   Latonya
4    w   0.0    Carrie
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 3 columns):
race         4870 non-null object
call         4870 non-null float32
firstname    4870 non-null object
dtypes: float32(1), object(2)
memory usage: 133.2+ KB
None


In [5]:
# number of callbacks for black and white sounding names
white_calls=sum(data[data.race=='w'].call)
black_calls=sum(data[data.race=='b'].call)
total_white_calls=data[data.race=='w'].race.count()
total_black_calls=data[data.race=='b'].race.count()
total_check=bool(total_white_calls+total_black_calls==data.race.count())

#print raw numbers:
print('white callbacks: ',white_calls)
print('total white applicants: ',total_white_calls)
print('black callbacks: ',black_calls)
print('total black applicants: ',total_black_calls)
print('totals add up: ',total_check)



white callbacks:  235.0
total white applicants:  2435
black callbacks:  157.0
total black applicants:  2435
totals add up:  True


1.)  Exploring the data we can see that the values from the columns are either one of 2 values.  This would fit the standard deffinition of a binomial distribution and since we have 4870 of each, the distribution will be approximately normal by the De Moivre–Laplace theorem.

<div class="span5 alert alert-success">
<h2>SOLUTION: What test is appropriate for this problem?</h2>

<p>This is an interesting question, and it can be argued that there are two possible correct answers (such is life). First note the wording of the question: does race have a *significant impact* on callback. This usually suggests a $\chi^2$-test, but the $\chi^2$-test requires *frequencies* rather than percentages. Interestingly, one of the code snippets above uses frequency so we will look at that in a bit.</p>

For now, a two-sample $z$-test:

$$z = \frac{\left( \hat{p}_W - \hat{p}_B \right) - 0}{\sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}}$$

where

$$\hat{p} = \frac{y_W + y_B}{n_W + n_B}$$

The null and alternate hypotheses:

$$H_0: p_B = p_W$$
$$H_A: p_B \ne p_W$$

CLT applies because we assume that the samples are representative of the population. The observations in each sample are assumed to be independent since the sample was drawn randomly.

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

n_w = len(w)
n_b = len(b)

prop_w = np.sum(w.call) / len(w)
prop_b = np.sum(b.call) / len(b)

prop_diff = prop_w - prop_b
phat = (np.sum(w.call) + np.sum(b.call)) / (len(w) + len(b))

z = prop_diff / np.sqrt(phat * (1 - phat) * ((1 / n_w) + (1 / n_b)))
pval = stats.norm.cdf(-z) * 2
print("Z score: {}".format(z))
print("P-value: {}".format(pval))

Z score: 4.108412152434346
P-value: 3.983886837585077e-05


2.)  The Null hypothesis is that both races recieve the same percentage of calls back.  The alternative hypothesis is that race does effect the call back rate.

In [22]:
moe = 1.96 * np.sqrt(phat * (1 - phat) * ((1 / n_w) + (1 / n_b)))
ci = prop_diff + np.array([-1, 1]) * moe
t_test, p_value = stats.ttest_ind(w.call,b.call,equal_var=False)
print("Margin of Error: {}".format(moe))
print("Confidence interval: {}".format(ci))
print('t-statistic: {}'.format(t_test))
print('p-value: {}'.format(p_value))

Margin of Error: 0.015281912310894095
Confidence interval: [ 0.01675094  0.04731477]
t-statistic: 4.114705290861751
p-value: 3.942941513645935e-05


In [31]:
#create bootstrapping function
def draw_bs_reps(data, func, size=1):
    bs_replicates=np.empty(size)
    for i in range(size):
        bs_replicates[i]=func(np.random.choice(data,size=len(data)))    
    return bs_replicates   

# Compute mean of all forces: mean_force
mean_call = np.mean(data.call)

# Generate shifted arrays
w_shifted = w.call - np.mean(w.call) + mean_call
b_shifted = b.call - np.mean(b.call) + mean_call

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_w = draw_bs_reps(w_shifted, np.mean, 10000)
bs_replicates_b = draw_bs_reps(b_shifted, np.mean, 10000)

# Get difference of means
bs_replicates = bs_replicates_w - bs_replicates_b
empirical_diff_means= np.mean(w.call)-np.mean(b.call)

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means)/len(bs_replicates)
print('p-value: {}'.format(p))




p-value: 0.0001


3.)  The margin of error came out to 0.0153, and a confidence interval of 0.0168 to 0.0473.  The frequentist approach gave us a t-statistic was 4.1147 and a p-value was 0.000039.  The p-value given by the bootstrapping test was 0.0001.

4.)  The two experiments tell us that race associated with certain names does indeed have an effect on the amount of call backs and requests for interviews.  The result so far does not say the severity, but only that it is significant and it exists.

5.)  As mentioned, these tests only state that it is a factor in the call back rate.  More research would need to be done in order to decide which factors have the biggest impact on call backs.

<div class="span5 alert alert-success">
<p>The p-value is practically 0 so we reject the null hypothesis that white and black sounding names have the same callback rate. They do not.</p>

<p>Since 0 is not in the confidence interval, we reject the null hypothesis with the same conclusion.</p>

<h4>SOLUTION: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</h4>

<p>No! While our test did show that there is a difference in callback rate based on race alone, there are other variables that may also contribute to, or interact with, race to explain the difference. In the original research paper, the researchers cited geography/city as a confounding variable. Additionally, we could also look at education and experience levels as well. But, in our very narrow example, we have shown that there is a significant difference in callback rates between white people and black people.</p>

<p>The $\chi^2$ approach:</p>
</div>

In [11]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.44902858418937
p-value: 4.997578389963255e-05


<div class="span5 alert alert-success">
The chi-squared test yields a similar result. We reject the null hypothesis that race and callback rate are independent. The margin of error and confidence interval calculations are a bit more complicated because the chi-squared distribution is not always symmetric, depending on the number of degrees of freedom.
</div>